<a href="https://colab.research.google.com/github/georgeto20/CIS-700-008/blob/master/Description_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Install HuggingFace Transfomers library.

In [0]:
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples/language-modeling')
os.chdir('/content/transformers/examples/text-generation')
!pip install dict_to_obj


In [0]:
import torch
import run_language_modeling
import run_generation
from dict_to_obj import DictToObj
import collections
import random
import numpy as np
from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pickle
!wget http://parl.ai/downloads/light/light-environment.pkl

In [0]:
with open('light-environment.pkl', 'rb') as f:
  env_data = pickle.load(f)

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

object_description_text = [[base_form, description] for obj in env_data['objects'].values() for (base_form, description) in zip(obj['base_form'], obj['descriptions'])]
character_description_text = [[obj['base_form'][0], obj['desc']] for obj in env_data['characters'].values()]
location_description_text = [[obj['setting'], obj['description']] for obj in env_data['rooms'].values()]

for (array, mode) in zip([object_description_text, character_description_text, location_description_text], ["object", "character", "location"]):
  train_array, test_array = train_test_split(array, test_size=0.1, random_state=42)
  valid_array, test_array = train_test_split(test_array, test_size=0.5, random_state=42)

  with open('/content/' + mode + '_description_train.txt', 'w') as train_txt:
    for line in train_array:
      train_txt.write("\n".join(line)+"\n")
    train_txt.close()

  with open('/content/' + mode + '_description_valid.txt', 'w') as valid_txt:
    for line in valid_array:
      valid_txt.write("\n".join(line)+"\n")
    valid_txt.close()

  with open('/content/' + mode + '_description_test.txt', 'w') as test_txt:
    for line in test_array:
      test_txt.write("\n".join(line)+"\n")
    test_txt.close()

## Finetune and Eval
The Hugging Face library provides a script [run_language_modeling.py](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) which contains all of the code for training and evaluating a language model.

We will be calling this script directly from the command line in order to launch training. We will also use functions from this script to conduct evaluation and generate samples at inference time.

### Launch fine-tuninng
We will be calling `run_language_modeling.py` from the command line to launch fine-tuning, **Running fine-tuning may take several hours.** Every `save_steps` steps, a checkpoint is saved to disk. The checkpoint contains all the learned weights for your model, and you can  always reload the model from a saved checkpoint, even if your Colab has crashed.

Below is an explanation of some of the arguments you might want to modify in the command below. 

* `--line_by_line`: Add `--line_by_line` if distinct lines of the text should be treated as distinct training examples. For example, if your dataset contains one story/tweet/article per line, this should be set.
* `--num_train_epochs`: The number of times to iterate over the train set. Increasing the number of epochs may result in better performance, but making this number too high will cause the model to overfit on the train set.
* `--block_size`: Your training text is truncated into blocks of this length. At test time, you will only want to generate sequences that are at most this length.
* `--gradient_accumulation_steps`: Update the model weights every this many steps. You shold set this to >1 when the batch size is very small to improve training stability.
* `--output_dir`: This is the where checkpoints will get saved. When you finetune on your own dataset, you should change this path. We recommend saving checkpoints to your Google Drive (`/content/drive/My Drive/`) so you can access them even if the Colab session dies.
* `--model_name_or_path` The path to the model weights to use when starting fine-tuning. You can set this to `gpt2-medium` to initialize with GPT-2's 355 million parameter model, or `gpt2` to initialize with their smaller 124 million parameter model. You can also set this to one of your own checkpoints to restart your training job if it crashes.

**I am getting out-of memory errors. What do I do?**

The number of trainable paramters in the model is a function of the `block_size` and the `batch_size`. If you are getting out-of-memory errors, then try drecreasing these value.

**Oh no! My computer went to sleep and the Colab disconnected.**

The train job might have still completed. Check the `output_dir` in your Google Drive to see if checkpoint files have been created there.

**Training is taking foreverrrrrr.**

Try decreasing `num_train_epochs` or changing `model_name_or_path` to `gpt2` instead of `gpt2-medium`.
If your evaluation set is very large, you might also want to remove the `evaluate_during_training` flag or increase `logging_steps`.

In [0]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/CIS700-project/finetuned_models/objects' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=500 \
    --train_data_file=/content/object_description_train.txt \
    --do_eval \
    --eval_data_file=/content/object_description_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=64 \
    --gradient_accumulation_steps=5

In [0]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/CIS700-project/finetuned_models/characters' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=500 \
    --train_data_file=/content/character_description_train.txt \
    --do_eval \
    --eval_data_file=/content/character_description_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=64 \
    --gradient_accumulation_steps=5

In [0]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/CIS700-project/finetuned_models/locations' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=500 \
    --train_data_file=/content/location_description_train.txt \
    --do_eval \
    --eval_data_file=/content/location_description_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=64 \
    --gradient_accumulation_steps=5

### Compute perplexity of a dataset.
This section shows how to compute perplexity of a dataset according to either the pre-trained or your fine-tuned language model. While this is possible to do by calling `run_language_modeling.py` on the command-line as above, we'll instead call the Python functions directly.

#### Helper functions

In [0]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  #config_class, model_class, _ = run_language_modeling.MODEL_CLASSES[args.model_type]
  #config = config_class.from_pretrained(args.model_name_or_path, cache_dir=None)
  config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=None)
  model = AutoModelWithLMHead.from_pretrained(args.model_name_or_path,from_tf=bool(".ckpt" in args.model_name_or_path), config=config, cache_dir=None)
  """
  model = model_class.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None,
  )
  """
  if not next(model.parameters()).is_cuda:
    model.to(args.device)
  return model

def set_seed(args, seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  #_, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  #tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)
  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)
  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

#### Compute it.

In [0]:
# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/CIS700-project/finetuned_models/env_data'
#CHECKPOINT_PATH = "gpt2"

# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/description_valid.txt",
              "/content/description_test.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObj(args)

model = load_model(args)

for data_path in DATA_PATHS:
  eval_results = do_perplexity_eval(args, model, data_path)
  perplexity = eval_results['perplexity']
  print('{} is the perplexity of {} according to {}'.format(
      perplexity, data_path, CHECKPOINT_PATH))

### Generate samples
The following code generates text samples that are are continuations of a provided prompt.

In [0]:
!pip3 install pymagnitude
!wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")

In [0]:
import logging
logger = logging.getLogger('transformers')
logger.setLevel(50)

In [0]:
def construct_sentence_vector(description, vectors):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  for word in description.split():
    word_vector = vectors.query(word)
    sentence_vector += word_vector
  sentence_vector /= len(description.split())
  return sentence_vector

def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args, args.seed)

  #_, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)
  #tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  best_sequence = ""
  max_similarity = 0
  constructed_user_description = construct_sentence_vector(prompt_text, vectors)
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
    print('\n====== GENERATION {} ======'.format(generated_sequence_idx))
    print(prompt_text, text)
    # Add the prompt at the beginning of the sequence.
    constructed_known_description = construct_sentence_vector(text, vectors)
    similarity = vectors.similarity(constructed_user_description, constructed_known_description)
    print("Similarity score: {}".format(similarity))
    if similarity > max_similarity:
      max_similarity = similarity
      best_sequence = text
  print('\nThe best generation for {} is: {}'.format(prompt_text, best_sequence))
  print('Its similarity score is: {}'.format(max_similarity))
  return best_sequence

In [0]:
#import nltk
#nltk.download('punkt')
#from nltk.tokenize import sent_tokenize
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
def generate_description(mode, prompt):
  CHECKPOINT_PATH = '/content/drive/My Drive/CIS700-project/finetuned_models/' + mode
  #CHECKPOINT_PATH = 'gpt2-medium'

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  n_gpu = torch.cuda.device_count()
  #print("Running on device: ", device)

  args = collections.defaultdict(
    model_name_or_path=CHECKPOINT_PATH,
    output_dir=CHECKPOINT_PATH,
    n_gpu=n_gpu,
    mlm=False,
    device=device,
    model_type='gpt2',
    seed=42,
    stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
    temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
    k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
    p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
    repetition_penalty=None,
    length=50,  # Number of tokens to generate.
    num_return_sequences=100,  # Number of independently computed samples to generate.
  )
  args = DictToObj(args)

  model = load_model(args)
  description = generate_samples(args, model, prompt)
  description_sentences = description.split('\n')
  best_sentence = ""
  max_length = 0
  for sentence in description_sentences:
    if len(sentence) > max_length:
      best_sentence = sentence
      max_length = len(sentence)
  best_sentence = best_sentence.lstrip(' ,.')
  print("\nThe final description for {} is: {}".format(prompt, best_sentence))
  torch.cuda.empty_cache()
  return best_sentence

In [0]:
generate_description("objects", "table")